In [3]:

from scipy.signal import convolve2d
import os
import io
import cv2
import glob
import time
import pickle
import shutil
import numpy as np
import traceback
import logging
import skimage.io as io
#from PIL import Image as PI
from wand.image import Image as WI
from wand.display import display
from pathlib import Path
from apputil import setup_logger
from segmenter.pre_processing import *
from imutils import resize as im_resize
from scipy.ndimage import binary_fill_holes
from skimage.morphology import skeletonize, thin
from skimage.filters import threshold_otsu, gaussian, median, threshold_yen
import numpy as np
import math
from skimage.restoration import estimate_sigma
from time import sleep

from wrapt_timeout_decorator import *
value=None

def pixel_ratio(bin_img):
    white_px = np.sum(img == 255)
    black_px = np.sum(img == 0)
    ratio = white_px // black_px
    return ratio, white_px, black_px

@timeout(20)
def long_task(img):
    import time as tim
    import numpy as np
    import io
    import cv2
    from PIL import Image
    from wand.image import Image as WI
    from segmenter.pre_processing import binarize_image
    img_buffer=None
    #th3 = cv2.adaptiveThreshold(img,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C,\ cv2.THRESH_BINARY,11,2)
    with WI.from_array(img) as im:
        im.deskew(0.4*im.quantum_range)
        pil_image = Image.open(io.BytesIO(im.make_blob("png")))                               
        np_image = np.array(pil_image)
        ret3, result = cv2.threshold(np_image,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        return result
        
def estimate_noise2(img):
    return estimate_sigma(img, multichannel=True, average_sigmas=True)

def estimate_noise(I):
    H, W = I.shape
    M = [[1, -2, 1],
         [-2, 4, -2],
         [1, -2, 1]]
    sigma = np.sum(np.sum(np.absolute(convolve2d(I, M))))
    sigma = sigma * math.sqrt(0.5 * math.pi) / (6 * (W-2) * (H-2))
    return sigma
def countpix(img):
    # get all non black Pixels
    
    wht = cv2.countNonZero(img)
    # get pixel count of image
    height, width = img.shape
    px = height*width
    # compute all black pixels
    blk = px - wht
    return wht, blk

def begin():
    raise ValueError("Test")
    img = cv2.imread('sent_images/test.jpeg')
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    try:
        result = long_task(img)
        #result = cv2.cvtColor(result, cv2.COLOR_BGR2GRAY)
    except Exception as e:
        print(e)
        #raise

    print('OpenCV: ', result.shape)
    cv2.imwrite('sent_images/bad_img.png', result)
    img_m = np.array(result)
    print('Checking ratio')
    # ratio,white_px,black_px = pixel_ratio(result)
    wht, blk = countpix(result)
    if (blk > wht):
        print('Too many black')
    print(f"{wht} {blk}")

    noise = estimate_noise(img_m)
    print(f"Manual Sigma: {noise}")
    noise = estimate_noise2(img_m)
    print(f"Scikit Sigma: {noise}")
    
try:
    begin()
except Exception as e:
    print(e)
#print(value)
# with WI.from_array(img) as im:
#     img_buffer = np.asarray(bytearray(im.make_blob("PNG")), dtype=np.uint8)
#     ret, mat = binarize_image(img_buffer)
#     with WI(blob=mat) as timg:
#         imgf = mat
#         timg.deskew(0.4*im.quantum_range)
#         imgf = np.array(timg)
#         img_buffer = np.asarray(bytearray(timg.make_blob("PNG")), dtype=np.uint8)
#         imgmat = cv2.imdecode(img_buffer, cv2.IMREAD_GRAYSCALE)
# cv2.imshow('Image',imgmat)
# cv2.waitKey(0)
# cv2.destroyAllWindows()
# imgmat = get_thresholded(imgmat, threshold_otsu(imgmat))


Test


In [39]:
def dep(val):
    val1 = val-1
    val2 = val+1
    val3 = None
    return val1, val2, val3
val = 3
val1, val2, val3 = dep(val)
print(val)
print(val1)
print(val2)
print(val3)

3
2
4
None


In [ ]:
# read your one blurred image and convert to gray
im_gray = cv2.imread('test/lena.png', 0)

# threshold it with OTSU thresholding and get the threshold value
thresh, im_bw = cv2.threshold(im_gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)
print(thresh)

# define the paths to your input images and to where you want to put the output images    
in_dir = 'test'
out_dir = 'test2'

# read the input image file names with paths into a list
infiles = in_dir + '/*.png'
img_names = glob(infiles)
print(img_names)

# loop over each input image in a for loop
for fn in img_names:
    print('processing %s...' % fn)

    # read an input image as gray
    im_gray = cv2.imread(fn, 0)

    # threshold it with your saved threshold
    im_bw = cv2.threshold(im_gray, thresh, 255, cv2.THRESH_BINARY)[1]

    # write the result to disk in the previously created output directory
    name = os.path.basename(fn)
    outfile = out_dir + '/' + name
    cv2.imwrite(outfile, im_bw)

In [ ]:
def Slice(cv_img):
    start_time = time.time()
    img_buffer = imgmat = None
    segmented_staves=[]
    log.info("SLICER: beginning binarization " + str(time.time() - start_time))
    try:
        with Image.from_array(cv_img) as im:
            img_buffer = np.asarray(bytearray(im.make_blob("PNG")), dtype=np.uint8)
            ret, mat = binarize_image(img_buffer)
            with Image(blob=mat) as timg:
                timg.deskew(0.4*im.quantum_range)
                img_buffer = np.asarray(bytearray(timg.make_blob("PNG")), dtype=np.uint8)
                imgmat = cv2.imdecode(img_buffer, cv2.IMREAD_GRAYSCALE)
    except cv2.error as e:
        log.error(traceback.format_exc())   
        log.error("CV: read error")
        return
    
    log.info("SLICER: beginning segmentation " + str(time.time() - start_time)) 
    imgmat = get_thresholded(imgmat, threshold_otsu(imgmat))
    segmenter = Segmenter(imgmat)
    imgs_with_staff = segmenter.regions_with_staff
    log.info("SLICER: beginning cropping " + str(time.time() - start_time)) 
    for i, img in enumerate(imgs_with_staff):
        log.info("SLICER: image++ in outputs " + str(time.time() - start_time)) 
        segmented_staves.append(crop(img))
    log.info("SLICER: work completed " + str(time.time() - start_time)) 
    return segmented_staves

if __name__ == '__main__':
    Slice(r"C:\Users\aroue\Downloads\Documents\@ML\Sheet Music\goodsheet\pgws.png")